###### [NOTE] The original input increments by 0.5, but for numerical stability I multiple all by 2, and round to the nearest integer. so now instead of 50*50*50, it is 100*100*100

In [ ]:
import sys, os, glob
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

pd.set_option('display.max_colwidth', -1)

In [ ]:
# you can later use this array to iterate through all files
files = [file for file in glob.glob("result/*.txt")]
print ("NUMBER OF FILES IN RESULT FOLDER: ", len(files))

###### FUNCTIONS

In [ ]:
def getBuildingHeights(filename):
    '''
    input: name of result file
    output: array of 3 heights of our target building (i.e [3,27,6])
    '''
    return filename.split("/")[-1].replace(".txt", "").split(",")

def createColumnMapper():
    mapper = {0: "x", 1:"y", 2:"z"}
    month = [calendar.month_name[i+1] for i in range(12)]
    for i in range(3, 3+12):
        mapper[i] = month[i-3]
    return mapper

def convertToDF(fname):
    '''
    input: name of result file
    output: pandas df
    '''
    with open(fname) as f:
        content = f.readlines()   
    content = np.array([line.replace("\n", "").split(",") for line in content[1:]])
    return pd.DataFrame(data=content)

def matplotlibViz(df):
    '''
    Visualizes in 3D a dataframe
    input: dataframe
    output: None
    '''
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df['x'], df['y'], df['z'])
    plt.title("3-D rendered building")
    plt.show()

def convertToGrid(df, size=(100,100,100), increment=1):
    '''
    zero pad all the coordinates that do not have radiation values (even the coordinates inside the building are zero padded).
    input: dataframe
    output: dataframe
    
    '''
    res = []
    temp = df_building[['x', 'y', 'z']]*2
    temp = temp.apply(pd.to_numeric, errors='coerce', downcast="integer")
    hashmap = {tuple(row) for index, row in temp.iterrows()}

    for x in range(0, size[0]+increment, increment):
        for y in range(0, size[0]+increment, increment):
            for z in range(0, size[0]+increment, increment):
                if (x,y,z) in hashmap:
                    res.append([x,y,z,1])
                else:
                    res.append([x,y,z,0])
    
    assert len(res) == (size[0]+1)**3
    return pd.DataFrame(data=res)

###### CONVERTED DATAFRAME

In [ ]:
FILE_NAME = "result/30,24,6.txt"
df = convertToDF(FILE_NAME)
df_building = df.rename(index=int, columns=createColumnMapper()).rename(index=int, columns={15:"Annual Sum"}).apply(pd.to_numeric, errors='coerce')
df_building.head(5)

###### PADDING

In [ ]:
convertToGrid(df_building).head()

###### MATPLOTLIB VISUALIZATION OF OUR TARGET BUILDING

In [ ]:
# 30,24,6 HEIGHTS
matplotlibViz(df_building)

###### MODEL PROTOTYPE

   Initial naive approach is to have two separate 3D-convolution for our target building and the boundary buildings. This will ensure that the two inputs (target, boundary) are separated in our architecture. After the 3D convolution, we could first try with a simple concatenation of the dense layer of the resulting outputs, then do some fc layers, then do a prediction coordinate by coordinate (since we have a boundary of 50 by 50 by 50).
    
   If this sort of works, then we could play around with integrating some sort of LSTM to do a monthly radiation prediction. I might be completely off about this (not sure what our end goal is), so please tell me if this sounds off.
    
   We could refer to https://gist.github.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2 for a basic idea of how to do a 3D CNN.